In [ ]:
import osmnx as ox, geopandas as gpd
import srtm as sr
import networkx as nx
from networkx.classes.coreviews import AtlasView, AdjacencyView
from networkx.classes.reportviews import NodeView, EdgeView, DegreeView
from networkx.exception import NetworkXError
import networkx.convert as convert
from networkx.utils import pairwise
%matplotlib inline
ox.config(log_file=True, log_console=True, use_cache=True)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from haversine import haversine
import math
from math import radians
from shapely.geometry import Point, LineString
from collections import deque
from heapq import heappush, heappop
from itertools import count
import networkx as nx
from networkx.utils import generate_unique_node
import time
import random
from sklearn.neighbors import KNeighborsClassifier
import gzip
import zipfile
import json, requests
import folium
import ast
import database_io as db
from statistics import mean, median
import trajectory as tj
# Capacità della batteria
eBatCap = 38*(3600*1e3)

### Functions to calculate the shortest path

In [ ]:
def peso(G, weight):
    """Returns a function that returns the weight of an edge.

    The returned function is specifically suitable for input to
    functions :func:`_dijkstra` and :func:`_bellman_ford_relaxation`.

    Parameters
    ----------
    G : NetworkX graph.

    weight : string or function
        If it is callable, `weight` itself is returned. If it is a string,
        it is assumed to be the name of the edge attribute that represents
        the weight of an edge. In that case, a function is returned that
        gets the edge weight according to the specified edge attribute.

    Returns
    -------
    function
        This function returns a callable that accepts exactly three inputs:
        a node, an node adjacent to the first one, and the edge attribute
        dictionary for the eedge joining those nodes. That function returns
        a number representing the weight of an edge.

    If `G` is a multigraph, and `weight` is not callable, the
    minimum edge weight over all parallel edges is returned. If any edge
    does not have an attribute with key `weight`, it is assumed to
    have weight one.

    """
    if callable(weight):
        return weight
    # If the weight keyword argument is not callable, we assume it is a
    # string representing the edge attribute containing the weight of
    # the edge.
    if G.is_multigraph():
        return lambda u, v, d: min(attr.get(weight, 1) for attr in d.values())
    return lambda u, v, data: data.get(weight, 1)


In [ ]:
def dijkstra_modified(G, sources, weight, battery, consumo, anxiety_value, pred=None, paths=None,
                          cutoff=None, target=None):
    """Uses Dijkstra's algorithm to find shortest weighted paths

    Parameters
    ----------
    G : NetworkX graph

    sources : non-empty iterable of nodes
        Starting nodes for paths. If this is just an iterable containing
        a single node, then all paths computed by this function will
        start from that node. If there are two or more nodes in this
        iterable, the computed paths may begin from any one of the start
        nodes.

    weight: function
        Function with (u, v, data) input that returns that edges weight

    pred: dict of lists, optional(default=None)
        dict to store a list of predecessors keyed by that node
        If None, predecessors are not stored.

    paths: dict, optional (default=None)
        dict to store the path list from source to each node, keyed by node.
        If None, paths are not stored.

    target : node label, optional
        Ending node for path. Search is halted when target is found.

    cutoff : integer or float, optional
        Depth to stop the search. Only return paths with length <= cutoff.

    Returns
    -------
    distance : dictionary
        A mapping from node to shortest distance to that node from one
        of the source nodes.

    Raises
    ------
    NodeNotFound
        If any of `sources` is not in `G`.

    Notes
    -----
    The optional predecessor and path dictionaries can be accessed by
    the caller through the original pred and paths objects passed
    as arguments. No need to explicitly return pred or paths.

    """
    G_succ = G._succ if G.is_directed() else G._adj

    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    batt = {}
    seen = {}
    seenbat = {}
    # fringe is heapq with 4-tuples (distance,c,node,battery)
    # use the count c to avoid comparing nodes (may not be able to)
    c = count()
    fringe = []
    
    for source in sources:
        #print("source:",source)
        if source not in G:
            raise nx.NodeNotFound("Source {} not in G".format(source))
        seen[source] = 0
        seenbat[source] = eBatCap
        push(fringe, (0, next(c), source, battery))
    while fringe:
        (d, _, v, b) = pop(fringe)
        if v in dist:
            continue  # already searched this node.
        dist[v] = d
        batt[v] = b
        if v == target:
            break
        for u, e in G_succ[v].items():
            cost = float(weight(v,u,e))
            consumption = float(consumo(v,u,e))
            
            if cost is None:
                continue
            vu_dist = dist[v] + cost
            vu_autonom = batt[v] - consumption
            if vu_autonom > eBatCap:
                vu_autonom = eBatCap
            if cutoff is not None:
                if vu_dist > cutoff:
                    continue
            if u in dist:
                if vu_dist < dist[u]:
                    raise ValueError('Contradictory paths found:','negative weights?')
            elif u not in seen or vu_dist < seen[u]:
                if vu_autonom - (eBatCap*anxiety_value) > 0:
                    seen[u] = vu_dist
                    seenbat[u] = vu_autonom
                    push(fringe, (vu_dist, next(c), u, vu_autonom))
                    if paths is not None:
                        paths[u] = paths[v] + [u]
                    if pred is not None:
                        pred[u] = [v]
            elif vu_dist == seen[u] and vu_autonom - (eBatCap*anxiety_value) > 0:
                if pred is not None:
                    pred[u].append(v)

    # The optional predecessor and path dictionaries can be accessed
    # by the caller via the pred and paths objects passed as arguments.
    return dist,batt

In [ ]:
# Anxiety_value is a float representing the minimum battery percentage available

def dijkstraMod (G, sources, battery = None, anxiety_value=None , target=None, cutoff=None, weight='weight', consumo='consumo'):
    if not sources:
        raise ValueError('sources must not be empty')
    if target in sources:
        return (0, target)
    weight = peso(G, weight)
    consumo = peso(G, consumo)
    if battery is None:
        battery = eBatCap
    else:
        battery = battery*(3600*1e3)
    if anxiety_value is None:
        anxiety_value = 0
    paths = {source: [source] for source in sources}  # dictionary of paths
    dist, batt = dijkstra_modified(G, sources, weight, battery, consumo, anxiety_value, paths=paths,
                                 cutoff=cutoff, target=target)

    if target is None:
        return (dist, batt, paths)
    else:
        try:
            return (dist[target], batt[target], paths[target])
        except KeyError:
#             print('Il nodo {} non è raggiungibile'.format(target))
            return (None, None, None)

In [ ]:
cap = 0
def dijkstra_modified_batt_noConstrain(G, sources, weight, battery, consumo, anxiety_value, pred=None, paths=None,
                          cutoff=None, target=None):
    """Uses Dijkstra's algorithm to find shortest weighted paths

    Parameters
    ----------
    G : NetworkX graph

    sources : non-empty iterable of nodes
        Starting nodes for paths. If this is just an iterable containing
        a single node, then all paths computed by this function will
        start from that node. If there are two or more nodes in this
        iterable, the computed paths may begin from any one of the start
        nodes.

    weight: function
        Function with (u, v, data) input that returns that edges weight

    pred: dict of lists, optional(default=None)
        dict to store a list of predecessors keyed by that node
        If None, predecessors are not stored.

    paths: dict, optional (default=None)
        dict to store the path list from source to each node, keyed by node.
        If None, paths are not stored.

    target : node label, optional
        Ending node for path. Search is halted when target is found.

    cutoff : integer or float, optional
        Depth to stop the search. Only return paths with length <= cutoff.

    Returns
    -------
    distance : dictionary
        A mapping from node to shortest distance to that node from one
        of the source nodes.

    Raises
    ------
    NodeNotFound
        If any of `sources` is not in `G`.

    Notes
    -----
    The optional predecessor and path dictionaries can be accessed by
    the caller through the original pred and paths objects passed
    as arguments. No need to explicitly return pred or paths.

    """
    G_succ = G._succ if G.is_directed() else G._adj

    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    batt = {}
    seen = {}
    seenbat = {}
    # fringe is heapq with 4-tuples (distance,c,node,battery)
    # use the count c to avoid comparing nodes (may not be able to)
    c = count()
    fringe = []
    for source in sources:
        if source not in G:
            raise nx.NodeNotFound("Source {} not in G".format(source))
        seen[source] = 0
        seenbat[source] = eBatCap
        push(fringe, (0, next(c), source, battery))
    while fringe:
        (d, _, v, b) = pop(fringe)
        if v in dist:
            continue  # already searched this node.
        dist[v] = d
        batt[v] = b
        if v == target:
            break
        for u, e in G_succ[v].items():
            cost = float(weight(v,u,e))
            consumption = float(consumo(v,u,e))
            if cost is None:
                continue
            vu_dist = dist[v] + cost
            vu_autonom = batt[v] - consumption
            if vu_autonom > cap:
                vu_autonom = cap
            if cutoff is not None:
                if vu_dist > cutoff:
                    continue
            if u in dist:
                if vu_dist < dist[u]:
                    raise ValueError('Contradictory paths found:','negative weights?')
            elif u not in seen or vu_dist < seen[u]:
                seen[u] = vu_dist
                seenbat[u] = vu_autonom
                push(fringe, (vu_dist, next(c), u, vu_autonom))
                if paths is not None:
                    paths[u] = paths[v] + [u]
                if pred is not None:
                    pred[u] = [v]
            elif vu_dist == seen[u]:
                if pred is not None:
                    pred[u].append(v)

    # The optional predecessor and path dictionaries can be accessed
    # by the caller via the pred and paths objects passed as arguments.
    return dist,batt

In [ ]:
# Anxiety_value is a float representing the minimum battery percentage available

def dijkstraMod_batt_noConstrain (G, sources, battery = None, anxiety_value=None , target=None, cutoff=None, weight='weight', consumo='consumo'):

    if not sources:
        raise ValueError('sources must not be empty')
    if target in sources:
        return (0, target)
    weight = peso(G, weight)
    consumo = peso(G, consumo)
    if battery is None:
        battery = eBatCap
    else:
        battery = battery*(3600*1e3)
    if anxiety_value is None:
        anxiety_value = 0
    paths = {source: [source] for source in sources}  # dictionary of paths
    dist, batt = dijkstra_modified_batt_noConstrain(G, sources, weight, battery, consumo, anxiety_value, paths=paths,
                                 cutoff=cutoff, target=target)
    if target is None:
        return (dist, batt, paths)
    else:
        try:
            return (dist[target], batt[target], paths[target])
        except KeyError:
            print('Il nodo {} non è raggiungibile'.format(target))
            return (None, None, None)

In [ ]:
def dijkstra_modified_noBatt(G, sources, weight, pred=None, paths=None,
                          cutoff=None, target=None):
    
    G_succ = G._succ if G.is_directed() else G._adj

    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    seen = {}
    # fringe is heapq with 3-tuples (distance,c,node)
    # use the count c to avoid comparing nodes (may not be able to)
    c = count()
    fringe = []
    for source in sources:
        
        if source not in G:
            raise nx.NodeNotFound("Source {} not in G".format(source))
        seen[source] = 0
        push(fringe, (0, next(c), source))
    while fringe:
        (d, _, v) = pop(fringe)
        if v in dist:
            continue  # already searched this node.
        dist[v] = d
        if v == target:
            break
        for u, e in G_succ[v].items():
            cost = float(weight(v, u, e))
            if cost is None:
                continue
            vu_dist = dist[v] + cost
            if cutoff is not None:
                if vu_dist > cutoff:
                    continue
            if u in dist:
                if vu_dist < dist[u]:
                    raise ValueError('Contradictory paths found:',
                                     'negative weights?')
            elif u not in seen or vu_dist < seen[u]:
                seen[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
                if paths is not None:
                    paths[u] = paths[v] + [u]
                if pred is not None:
                    pred[u] = [v]
            elif vu_dist == seen[u]:
                if pred is not None:
                    pred[u].append(v)

    # The optional predecessor and path dictionaries can be accessed
    # by the caller via the pred and paths objects passed as arguments.
    return dist

In [ ]:
def dijkstra_no_batt  (G, sources, target=None, cutoff=None, weight='weight'):

    if not sources:
        raise ValueError('sources must not be empty')
    if target in sources:
        return (0, [target])
    weight = peso(G, weight)
    paths = {source: [source] for source in sources}  # dictionary of paths
    dist = dijkstra_modified_noBatt(G, sources, weight, paths=paths,
                                 cutoff=cutoff, target=target)
    if target is None:
        return (dist, paths)
    else:
        try:
            return (dist[target], paths[target])
        except KeyError:
            print('Il nodo {} non è raggiungibile'.format(target))
            return (None,None)

### Loads the map

In [ ]:
def caricaMappaDaDisco (fileName,folder):
    G = ox.save_load.load_graphml(fileName, folder=folder)
    return G

filename = 'toscana_torrette_completo.graphml'

G = caricaMappaDaDisco(filename,None)


### Creates the subgraph using Dijkstra for electric vehicles, chosing a battery value also means chosing the amount battery charged everytime the car stops.

In [ ]:
eBatCap = 38*(3600*1e3)

def Crea_lista_sottografo_torrette_dijkstra (G, battery = None, anxiety_value = None, torrette = None):
    start = time.time()
    listaTorrette = []
    percorsi = {}
    
    if battery is None:
        battery = eBatCap
    else:
        battery = battery*(3600*1e3)
    if anxiety_value is None:
        anxiety_value = 0
    if torrette is None:
        for i in G.nodes():
            if 'tipo' in G.nodes[i]:
                listaTorrette.append(i)
        torrette = listaTorrette
    conta = 0
    for j in torrette:
        conta+=1
        dist,batt,path = dijkstraMod (G, [j], weight = 'traveltime', consumo = 'consumption')
        listaj = {}
        for t in torrette:
            if t in dist and t != j:
                listaj.update({t : (dist[t], batt[t], path[t])})               
        percorsi.update({j : listaj})
        print(conta)
    end = time.time()
    print(end-start)
        
    return percorsi

### Some nodes have an empty adjacency list, this part connects theese nodes

In [ ]:
vuoti = []
for nodo in G.nodes():     
    if not G[nodo]:
        vuoti.append(nodo)
        
for nodo in G.nodes():
    for vuoto in vuoti:
        if vuoto in G[nodo]:
            G.add_edge(vuoto,nodo)

### Creates a list of the subgraph

In [ ]:
# UNCOMMENT TO RUN

# import pickle 

# listaSottografo_dijkstra = Crea_lista_sottografo_torrette_dijkstra(G)
# filehandler = open('Lista_sottografo_dijkstra.obj', 'wb') 
# pickle.dump(listaSottografo_dijkstra, filehandler)

### Loads the list from the hard disk

In [ ]:
import pickle 
filehandler = open('Lista_sottografo_dijkstra.obj', 'rb') 
listaSottografo_dijkstra = pickle.load(filehandler)

In [ ]:
torri = []
for i in G.nodes():
    if 'tipo' in G.nodes[i]:
        torri.append(i)

### Creates the subgraph

In [ ]:
def crea_sottografo_torrette (Graph, listasottografo, G = None):
    if G is None:
        return print('Mancano le torrette')
    for i in G.nodes():
        if 'tipo' in G.nodes[i]:
            try:
                Graph.add_node(i, y = G.nodes[i]['y'], x = G.nodes[i]['x'], osmid = G.nodes[i]['osmid'] , conInf= G.nodes[i]['conInf'])
            except KeyError :
                Graph.add_node(i, y = G.nodes[i]['y'], x = G.nodes[i]['x'], osmid = G.nodes[i]['osmid'])
    for nodoA in Graph.nodes():
        try:
            for nodoB in listasottografo[nodoA].keys():
                Graph.add_edge(nodoA, nodoB, length = nodoB, traveltime =listasottografo[nodoA][nodoB][0], batteria = listasottografo[nodoA][nodoB][1], percorso = listasottografo[nodoA][nodoB][2] )
        except KeyError :
            continue
    return Graph

In [ ]:
Gsub_D = nx.MultiDiGraph()
Gsub_D = crea_sottografo_torrette(Gsub_D, listaSottografo_dijkstra, G)


### Finds the shortest path using an heuristic 

In [ ]:
def aggiungi_source (Graph, nodo,battery = None, G = None):
#     check1s = time.time()
#     print("inizio aggiunta source ", check1s)
    if G is None:
        return print('Manca il grafo')
    dist,batt,path = dijkstraMod (G, [nodo],battery = battery, weight = 'traveFinds the shortest path using an heuristic¶ltime', consumo = 'consumption')
    Graph.add_node(nodo, y = G.nodes[nodo]['y'], x = G.nodes[nodo]['x'], osmid = G.nodes[nodo]['osmid'])
    for nodoB in Graph.nodes():
#         check2s = time.time()
        if nodoB in dist:
            Graph.add_edge(nodo,nodoB, length = nodoB,traveltime =dist[nodoB],batteria = batt[nodoB], percorso = path[nodoB])
#         check2e = time.time()
#         print(check2e-check2s)
#     check1e = time.time()
#     print("fine aggiunta source " ,check1e-check1s)

def aggiung_target_eur1(Graph,lista_torrette,partenza, nodo,G = None):
    check3s = time.time()
#     print("inizio aggiunta target ", check3s)
    Graph.add_node(nodo, y = G.nodes[nodo]['y'], x = G.nodes[nodo]['x'], osmid = G.nodes[nodo]['osmid'])
    for nodoA in lista_torrette:
#         print(nodoA)
        check4s = time.time()
        if nodoA != nodo and nodoA != partenza:
            dist,batt,path = dijkstraMod (G, [nodoA],target = nodo, weight = 'traveltime', consumo = 'consumption')
            if dist is not None:
                Graph.add_edge(nodoA,nodo, length = nodo,traveltime =dist,batteria = batt, percorso = path)
        check4e = time.time()
#         print(check4e-check4s)
    check3e = time.time()
#     print("fine aggiunta target ",check3e-check3s)

In [ ]:
def euristica3Knn(G,subG,nodi_scelti,batteria):
    percorso_tempo_batt_eur3 = {}
    start = time.time()
    partenza = nodi_scelti[0]
    arrivo = nodi_scelti[1]
    #print(partenza, arrivo)
    if partenza != arrivo:
        tempGsub_D = subG.copy(as_view=False) 
        dist,batt,path = dijkstraMod (G, [partenza],target = arrivo, battery = batteria, weight = 'traveltime', consumo = 'consumption')
        if dist is None:
            dist,path = dijkstra_no_batt(G,[partenza],arrivo,weight='traveltime')
            if dist is not None:
                torrette_check_end = []
                node_twr = list(subG.nodes)
                node_xy = [ [subG.nodes[id]['x'],subG.nodes[id]['y']] for id in node_twr ]
                model = KNeighborsClassifier(metric='euclidean', n_neighbors=1)
                model.fit(node_xy, node_twr)
                pred2 = model.predict([[G.nodes[arrivo]['x'],G.nodes[arrivo]['y']]])[0]
                if(pred2 not in torrette_check_end):
                    torrette_check_end.append(pred2)
#                 print("proviamo ",len(torrette_check), " torrette")
                aggiungi_source (tempGsub_D,partenza,battery = batteria, G = G)
                aggiung_target_eur1(tempGsub_D,torrette_check_end,partenza, arrivo,G)
                dist,batt,path = dijkstraMod_batt_noConstrain(tempGsub_D, [partenza],target = arrivo, battery = 0, weight = 'traveltime',  consumo = 'batteria')
                real_path = []
                for stop in range(0 , len(path)-1):
                    temp_path = tempGsub_D[path[stop]][path[stop+1]][0]['percorso']
                    real_path.append(temp_path)
                end = time.time()
                tempo = end-start
                percorso_tempo_batt_eur3[partenza,arrivo]=(dist,batt,path,real_path,tempo,"charge")
        else:
            batt = batt/(3600*1e3)
            end = time.time()
            tempo = end-start
            percorso_tempo_batt_eur3[partenza,arrivo]=(dist,batt,path,tempo)  
    return percorso_tempo_batt_eur3